# Network Operations

In this notebook we will assemble our project.  We will explore different functions on our dataset and compile them into a workflow ready for production.

The functions we will use will be a mix of `hub` based functions from our [MLRun Functions](http://github.com/mlrun/functions) repo, local and git based notebooks.

> The notebook should be run after generating the data in the [Generator Notebook](generator.ipynb)


we will start by setting up our environment, Loading MLRun and some utilities we will need

In [1]:
# Utils
import os
import json
import urllib
import numpy as np1231fdsafsafdas

# MLRun imports
from mlrun import mlconf
import kfp

# Setup API Endpoint
mlconf.dbpath = 'http://mlrun-api:8080'

# Set MLRun version for imges
MLRUN_COMMIT='v0.4.6'

Now lets define our current project 

## Create a project from a git repository

In [72]:
from mlrun import new_project

# update the dir and repo to reflect real locations 
# the remote git repo must be initialized in GitHub
project_dir = '../'
remote_git = 'https://github.com/zilbermanor/demo-network-operations.git'

# Create the project
newproj = new_project('network-operations', project_dir, init_git=True)

# We can update our project directory to the latest status by running
# newproj.pull()

Now that we have our project directory, lets forword our artifacts there to keep track of them

In [73]:
# Define an artifact path to keep track of where our artifacts are going
ARTIFACT_PATH =  os.path.join(project_dir, 'artifacts', '{{run.uid}}')
mlconf.artifact_path = os.path.join(project_dir, 'artifacts', '{{run.uid}}')

## Create and run functions

As we receive a new dataset, the first thing we would like to do is to explore it a bit, we can do that using our `describe` function in `mlrun/functions`

In [74]:
from mlrun import mount_v3io, new_model_server

In [75]:
# Define function versions to take
describe_tag = 'development'
train_tag = 'refacs'

In [76]:
# Import the functions

# Nuclio fuction from Notebook
newproj.set_function(func='notebooks/generator.ipynb',
                     name='generator')

# Kubernetes Job from Notebook
newproj.set_function(func='notebooks/preprocessing.ipynb', 
                     name='aggregate')

# Functions From hub
newproj.set_function(func=f'hub://describe:{describe_tag}', 
                     name='describe')

newproj.set_function(func=f'hub://sklearn_classifier:{train_tag}', 
                     name='train')

newproj.set_functionI(func=f'hub://test_model:{test_tag}',
                      name='test')

# Nuclio based Model Server From local notebook
newproj.set_function(func='notebooks/model-server.ipynb',
                     name="serving")

In [77]:
newproj.functions

[{'url': 'notebooks/generator.ipynb', 'name': 'generator'},
 {'url': 'notebooks/preprocessing.ipynb', 'name': 'aggregate'},
 {'url': 'hub://describe:development', 'name': 'describe'},
 {'url': 'hub://sklearn_classifier:refacs', 'name': 'train'},
 {'url': 'notebooks/model-server.ipynb', 'name': 'serving'}]

## Workflows:

**Utils**
Configuration file

Streaming: Generator (nuclio) -> Aggregate (nuclio) -> Inference (nuclio)
           -> Grafana dashboard

Training: Generator -> Aggregate -> Train -> Inference
           -> Kubeflow Pipeline

## Generate the dataset
If needed go to ...

## Run the functions locally to develop the workflow

now we can **Run** the function locally on our sample data, we would like to get some details on our `raw` data

## Register raw data as project level artifact

### WIP

In [88]:
# Define base Dataset
newproj.log_artifact('metrics',
                     local_path = os.path.join('/User', 'demo-network-operations', 'data', 'metrics.pq'))

[mlrun] 2020-03-25 13:19:58,720 log artifact metrics at /User/demo-network-operations/data/metrics.pq, size: 25500, db: Y


In [91]:
mount_v3io?

Signature:
mount_v3io(
    name='v3io',
    remote='~/',
    mount_path='/User',
    access_key='',
    user='',
    secret=None,
)
Docstring:
Modifier function to apply to a Container Op to volume mount a v3io path
Usage:
    train = train_op(...)
    train.apply(mount_v3io(container='users', sub_path='/iguazio', mount_path='/data'))
File:      ~/.pythonlibs/jupyter-2/lib/python3.6/site-packages/mlrun/platforms/iguazio.py
Type:      function


In [78]:
from mlrun import NewTask

In [83]:
describe_task = NewTask(
    name="describe", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": "is_error", 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots'},
    inputs={"table": 'store:///metrics'},
    artifact_path=ARTIFACT_PATH)

In [84]:
decsribe_run = newproj.func('describe').apply(mount_v3io()).run(describe_task)

[mlrun] 2020-03-25 13:18:42,935 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-03-25 13:18:42,936 starting run describe uid=3682663ce2bf48e3bcd52d3f50f1ab5b  -> http://mlrun-api:8080
[mlrun] 2020-03-25 13:18:43,015 Job is running in the background, pod: describe-7lfcw
[mlrun] 2020-03-25 13:18:50,556 artifact path is not defined or is local, artifacts will not be visible in the UI
Traceback (most recent call last):
  File "/opt/conda/bin/mlrun", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 956, in invoke
    r

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...f1ab5b,0,Mar 25 13:18:42,error,describe,kind=jobowner=admin,table,"class_labels=['0', '1']key=summarylabel_column=is_errorplot_dest=plotsplot_hist=True",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3682663ce2bf48e3bcd52d3f50f1ab5b --project network-operations , !mlrun logs 3682663ce2bf48e3bcd52d3f50f1ab5b --project network-operations
[mlrun] 2020-03-25 13:18:52,154 run executed, status=error
runtime error: error, check logs


RunError: error, check logs

### Create the feature vector

We will use our [Aggregate](preprocessing.ipynb) function to create rolling window features for our feature vector.

In doing so we hope that we could help our algorithms identify local errors by using a windowed trend

> Since this is a local function, if you didn't run the Aggregate notebook and deployed the function you will need to deploy it now

In [139]:
# Deploy aggregate function
# newproj.func('aggregate').deploy()

[mlrun] 2020-03-25 11:07:37,311 running build to add mlrun package, set with_mlrun=False to skip if its already in the image
[mlrun] 2020-03-25 11:07:37,311 starting remote build, image: .mlrun/func-network-operations-aggregate-latest


True

In [152]:
# Define aggregate task
aggregate_task = NewTask(
    name='aggregate',
    params={'df_artifact': os.path.join(project_dir, 'data', 'metrics.pq'),
            'metrics': ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
            'labels': ['is_error'],
            'metric_aggs': ['mean', 'sum'],
            'label_aggs': ['max'],
            'suffix': 'daily',
            'append_to_df': True,
            'window': 5,
            'center': True,
            'save_to': os.path.join(project_dir, 'data', 'aggregate.pq')},
    handler='aggregate')

newproj.func('aggregate').spec.image = 'mlrun/mlrun:0.4.6'
print(newproj.func('aggregate').to_yaml())

kind: job
metadata:
  name: aggregate
  tag: ''
  hash: ca53357c4713fa681b7340f3450a1e820a268813
  project: network-operations
  categories: []
spec:
  command: /User/demo-network-operations/src/aggregate.py
  args: []
  image: mlrun/mlrun:0.4.6
  volumes: []
  volume_mounts: []
  env:
  - name: V3IO_API
    value: ''
  - name: V3IO_USERNAME
    value: ''
  - name: V3IO_ACCESS_KEY
    value: ''
  description: ''
  build:
    image: .mlrun/func-network-operations-aggregate-latest
    commands: []
    code_origin: http://github.com/zilbermanor/demo-network-operations#b54282b5dff9dd4ec19e8d7a9f97adeb350478cf
status:
  state: ready
  build_pod: ''



In [168]:
newproj.func('aggregate').apply(mount_v3io()).run(aggregate_task)

[mlrun] 2020-03-25 11:27:55,239 starting run aggregate uid=1fc53a1e3318435a8a665345b79a3198  -> http://mlrun-api:8080
[mlrun] 2020-03-25 11:27:55,290 Job is running in the background, pod: aggregate-sr27r


KeyboardInterrupt: 

In [ ]:
from mlrun import new_function, NewTask, code_to_function, new_model_server

## Setup functions

### Test describe

In [10]:
summ_task = NewTask(
    "sum", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": "is_error", 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots'},
    inputs={"table": os.path.join(project_dir, 'data', 'aggregate.pq')},
    artifact_path=ARTIFACT_PATH)

In [11]:
summ_run = newproj.func('describe').run(summ_task)

[mlrun] 2020-03-25 09:17:58,817 starting run sum uid=bce3fa04af4d468998d5bbecc84eba68  -> http://mlrun-api:8080
[mlrun] 2020-03-25 09:17:58,901 Job is running in the background, pod: sum-lqlsx
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...4eba68,0,Mar 25 09:18:08,completed,sum,host=sum-lqlsxkind=jobowner=admin,table,"class_labels=['0', '1']key=summarylabel_column=is_errorplot_dest=plotsplot_hist=True",scale_pos_weight=15.73,histogramsimbalancecorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run bce3fa04af4d468998d5bbecc84eba68 --project network-operations , !mlrun logs bce3fa04af4d468998d5bbecc84eba68 --project network-operations
[mlrun] 2020-03-25 09:18:33,235 run executed, status=completed


In [43]:
model_configs = {
    "CLASS" : {
        "boosting_type"      : "gbdt",
        "num_leaves"         : 300,
        "max_depth"          : 50,
        "learning_rate"      : 0.1,
        "n_estimators"       : 300,
        "objective"          : "binary",
        "scale_pos_weight"   : 1,    
        "min_split_gain"     : 0.0,
        "min_child_samples"  : 20,
        "subsample"          : 1,
        "colsample_bytree"   : 1,
        "reg_alpha"          : 0,
        "reg_lambda"         : 1,
        "n_jobs"             : 16,
        "silent"             : True,
        "importance_type"    : "split",
        "random_state"       : 1},
    "FIT" : {
        "verbose"               : False
    },
    "META" : {
        "class" : "lightgbm.sklearn.LGBMClassifier",
        "version" : "2.3.1"
    }
}
display(model_configs)

{'CLASS': {'boosting_type': 'gbdt',
  'num_leaves': 300,
  'max_depth': 50,
  'learning_rate': 0.1,
  'n_estimators': 300,
  'objective': 'binary',
  'scale_pos_weight': 1,
  'min_split_gain': 0.0,
  'min_child_samples': 20,
  'subsample': 1,
  'colsample_bytree': 1,
  'reg_alpha': 0,
  'reg_lambda': 1,
  'n_jobs': 16,
  'silent': True,
  'importance_type': 'split',
  'random_state': 1},
 'FIT': {'verbose': False},
 'META': {'class': 'lightgbm.sklearn.LGBMClassifier', 'version': '2.3.1'}}

## Test train_lgbm

In [50]:
train_params = {
    "name" : 'train',
    "params" : {
        
        # CHOOSE YOUR MODEL
        'model_pkg_class' : config_filepath,
        'model_key'       : "models",
        
        # POINT THIS TO YOUR DATA
        'data_key'        : os.path.join(project_dir, 'data', 'aggregate_no_na.pq'),
        'sample'          : -1,
        'label_column'    : "is_error",
        'test_size'       : 0.10,
        'train_val_split' : 0.75,
        'rng'             : 1,
    
        # CHANGE ANY sklearn LogisticRegression class params here
        'class_params_updates'   : {
            'random_state' : 1,
        },
        
        # CHANGE ANY sklearn LogisticRegression fit params here
        'fit_params_updates'    : {}},
    
    "artifact_path" : f'{os.path.dirname(os.getcwd())}/artifacts'}


In [51]:
trainer = newproj.func('train').run(**train_params)

[mlrun] 2020-03-25 09:43:56,103 starting run train uid=65d1fff875ad4d85ba8d5e55bb39810f  -> http://mlrun-api:8080
[mlrun] 2020-03-25 09:43:56,252 Job is running in the background, pod: train-wkdkp
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
No handles with labels found to put in legend.
[mlrun] 2020-03-25 09:44:06,143 log artifact test_set at /User/demo-network-operations/artifacts/test_set.parquet, size: None, db: Y
[mlrun] 2020-03-25 09:44:06,252 log artifact models at /User/demo-network-operations/artifacts/models, size: None, db: Y
[mlrun] 2020-03-25 09:44:06,255 y_score.shape (124, 2)
[mlrun] 2020-03-25 09:44:06,255 yvalidb.shape (12

to track results use .show() or .logs() or in CLI: 
!mlrun get run 65d1fff875ad4d85ba8d5e55bb39810f --project network-operations , !mlrun logs 65d1fff875ad4d85ba8d5e55bb39810f --project network-operations
[mlrun] 2020-03-25 09:44:16,181 run executed, status=completed


## Pipeline

In [52]:
trainer.outputs

{'accuracy': 0.9758064516129032,
 'avg_precscore': 0.7272727272727273,
 'f1_score': 0.9758064516129032,
 'rocauc': 0.9870689655172414,
 'test_set': '/User/demo-network-operations/artifacts/test_set.parquet',
 'models': '/User/demo-network-operations/artifacts/models',
 'roc': '/User/demo-network-operations/artifacts/plots/roc.png',
 'confusion': '/User/demo-network-operations/artifacts/plots/confusion.png'}

In [ ]:
TODO: Take out the 

In [92]:
%%writefile project/workflow.py

def init_functions

@kfp.dsl.pipeline(
    name='Network Operations Demo',
    description='Train a Failure Prediction LGBM Model over sensor data'
)
def kfpipeline(
        df_artifact = os.path.join(project_dir, 'data', 'metrics.pq'),
        metrics = ['cpu_utilization'],
        labels = ['is_error'],
        metric_aggs = ['mean', 'sum'],
        label_aggs = ['max'],
        suffix = 'daily',
        append_to_df = True,
        window = 5,
        center = True,
        save_to = os.path.join(project_dir, 'data', 'aggregate.pq'),
        describe_table = 'summary',
        label_column = 'is_error',
        class_labels = [1, 0],
        SAMPLE_SIZE      = -1, # -n for random sample of n obs, -1 for entire dataset, +n for n consecutive rows
        TEST_SIZE        = 0.1,       # 10% set aside
        TRAIN_VAL_SPLIT  = 0.75,      # remainder split into train and val
        RNG              = 1,
        config_filepath = config_filepath,
    ):
    
    # Run preprocessing on the data
    aggregate = newproj.func('aggregate').as_step(name='aggregate',
                                                  params={'df_artifact': df_artifact,
                                                          'metrics': metrics,
                                                          'labels': labels,
                                                          'metric_aggs': metric_aggs,
                                                          'label_aggs': label_aggs,
                                                          'suffix': suffix,
                                                          'append_to_df': append_to_df,
                                                          'window': window,
                                                          'center': center,
                                                          'save_to': save_to},
                                                  outputs=['aggregate'],
                                                  handler='aggregate',
                                                  image='docker-registry.default-tenant.app.cnyidfihnjsz.iguazio-cd0.com:80/mlrun/func-default-aggregate-latest')

    describe = newproj.func('describe').as_step(name='describe',
                                                handler="summarize",  
                                                params={"key": "summary", 
                                                        "label_column": "is_error", 
                                                        'class_labels': ['0', '1'],
                                                        'plot_hist': True,
                                                        'plot_dest': 'plots'},
                                                inputs={"table": aggregate.outputs['aggregate']},
                                                outputs=["summary", "scale_pos_weight"])
    
    train = newproj.func('train').as_step(name='train', 
                                          handler='train_model',
                                          params={'model_pkg_class' : config_filepath,
                                                  'sample'          : -1,
                                                  'label_column'    : "is_error",
                                                  'test_size'       : 0.10,
                                                  'train_val_split' : 0.75,
                                                  'rng'             : 1},
                                          inputs={"data_key": aggregate.outputs['aggregate']},
                                          outputs=['model', 'test-set'])
    
    test = newproj.func('test').as_step()
    
    # deploy the model using nuclio functions
    deploy = newproj.func('serving').deploy_step(project='nuclio-serving',
                                                 models={'predictor': train.outputs['model']})

In [93]:
# for debug generate the pipeline dsl
kfp.compiler.Compiler().compile(kfpipeline, os.path.join(project_dir, 'yaml', 'lgbm_pipeline.yaml'))

/User/.pythonlibs/jupyter-2/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['cpu_utilization']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/User/.pythonlibs/jupyter-2/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['is_error']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/User/.pythonlibs/jupyter-2/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['mean', 'sum']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/User/.pythonlibs/jupyter-2/lib/python3.6/site-packages/kfp/components/_data_passing.py:16

In [94]:
mlrun.run_pipeline(kfpipeline, arguments={}, artifact_path=ARTIFACT_PATH, experiment='network-operations')

[mlrun] 2020-03-25 10:13:09,199 Pipeline run id=12700e49-b9ce-466c-83a2-d6d3818c9cdd, check UI or DB for progress


'12700e49-b9ce-466c-83a2-d6d3818c9cdd'

## Test endpoint

In [1]:
import pandas as pd
import requests
import json

In [2]:
# Set model
model_name = 'predictor'

# Load pre-processed data example
df = pd.read_parquet('/User/demo-network-operations/data/aggregate.pq')

# Set sample
sample = df.head(1).fillna(0).drop(columns=['is_error']).values.tolist()
msg = {'instances': sample}

# Set endpoint
addr = 'http://3.136.215.154:31092'

In [16]:
# Send Request
req = requests.post(f'{addr}/{model_name}/predict', data=json.dumps(msg))
req.__dict__

{'_content': b'[0.0]',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Server': 'nuclio', 'Date': 'Fri, 13 Mar 2020 10:25:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '5'},
 'raw': <urllib3.response.HTTPResponse at 0x7fb5137cd128>,
 'url': 'http://3.136.215.154:31092/predictor/predict',
 'encoding': None,
 'history': [],
 'reason': 'OK',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(0, 1, 156261),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7fb51332c908>}

## Add workflow

In [24]:
newproj.set_workflow('main', os.path.join(newproj.context, 'src', 'workflow.py'))

## Save Project yaml

In [26]:
newproj.save(os.path.join(newproj.context, 'project.yaml'))

In [22]:
newproj.context

'/User/demo-network-operations'